In [3]:
# sys
import os
# data structures
import pandas as pd
# Computation
import numpy as np
# OWN
import py_to_psql as pp
import psycopg2 as p

In [5]:
csv_path = os.getcwd() + os.path.join("\\Filtered_data") 
sliced_litho_logs_df = pd.read_csv(
    csv_path + os.path.join("\\sliced_litho_logs.csv")
)
sliced_logs = pd.read_csv(
    csv_path + os.path.join("\\sliced_logs.csv")
)
sliced_litho = pd.read_csv(
    csv_path + os.path.join("\\sliced_litho.csv")
)
dropped_wells = [
    '41/10- 1', '41/10a- 2Z', '41/14- 1', '41/15- 1', '41/20- 1', '41/25a- 1',
    '42/16- 1', '42/18- 2','42/21- 1', '42/22- 1', 
    '43/03- 1', '43/05- 1', 
    '44/12- 1'
]
# Dropping wells
sliced_litho_logs_df = sliced_litho_logs_df[~sliced_litho_logs_df.well_name.isin(dropped_wells)]
sliced_logs = sliced_logs[~sliced_logs.well_name.isin(dropped_wells)]
sliced_litho = sliced_litho[~sliced_litho.well_name.isin(dropped_wells)]

C:\Program Files\OpendTect\Python\envs\odmlpython-cpu-mkl\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [112]:
cell_width = 1
upscaled_df = pd.DataFrame(columns=[
    "well_name", "md_in_m", "gr", "dtco", "dtsh", "rhob", "nphi", "drho", "pef"
])
upscale_index = 0
for well_name in sliced_logs.well_name.unique():
    df = sliced_logs[sliced_logs["well_name"] == well_name]
    min_depth = df.min()["md_in_m"]
    max_depth = df.max()["md_in_m"]
    # Upscaling
    for depth in np.arange(min_depth, max_depth, cell_width):
        upscaled_values = df.loc[
            (df["md_in_m"] >= depth) 
            & (df["md_in_m"] < depth + cell_width)
        ].drop(columns=["Unnamed: 0", "md_in_m", "md_round2", "well_name"]).mean()
        upscaled_df.loc[upscale_index] = [well_name, depth] + upscaled_values.to_list()
        upscale_index += 1

In [113]:
upscaled_df

,well_name,md_in_m,gr,dtco,dtsh,rhob,nphi,drho,pef
0,20/06- 1,2403.5004,24.641343,51.082114,NaN,2.888900,0.010657,-0.003086,5.277757
1,20/06- 1,2404.5004,48.220600,67.968043,NaN,2.603471,0.179229,0.088014,4.982043
2,20/06- 1,2405.5004,39.176983,62.292583,NaN,2.593867,0.151950,0.072717,5.379117
3,20/06- 1,2406.5004,20.215143,51.468657,NaN,2.957514,-0.001857,-0.005771,6.286829
4,20/06- 1,2407.5004,19.381050,51.460083,NaN,2.958650,-0.007383,-0.011550,5.645400
...,...,...,...,...,...,...,...,...,...
10678,39/11- 1,2753.9111,32.077900,67.424871,NaN,NaN,NaN,NaN,NaN
10679,39/11- 1,2754.9111,27.424500,69.120729,NaN,NaN,NaN,NaN,NaN
10680,39/11- 1,2755.9111,26.087967,70.431367,NaN,NaN,NaN,NaN,NaN
10681,39/11- 1,2756.9111,27.753071,68.326457,NaN,NaN,NaN,NaN,NaN


In [114]:
a = sliced_logs[sliced_logs["well_name"] == sliced_logs.well_name.unique()[0]]
a_1 = a.loc[(a["md_in_m"] >= 2403.5004) & (a["md_in_m"] < 2403.5004 + 1)].drop(columns=["Unnamed: 0", "md_in_m", "md_round2", "well_name"])
a_1.mean()

gr      24.641343
dtco    51.082114
dtsh          NaN
rhob     2.888900
nphi     0.010657
drho    -0.003086
pef      5.277757
dtype: float64

In [77]:
aa = pd.DataFrame([])
aa["md"] = [22]

In [79]:
aa

,md
0,22


In [96]:
aa[a_1.columns] = a_1.mean()

In [97]:
aa

,md,gr,dtco,dtsh,rhob,nphi,drho,pef
0,22,24.641343,51.082114,NaN,2.8889,0.010657,-0.003086,5.277757


In [108]:
aa.loc[10] = [69,2,3,4,5,6,7,8]

In [109]:
aa

,md,gr,dtco,dtsh,rhob,nphi,drho,pef
0,22,24.641343,51.082114,NaN,2.8889,0.010657,-0.003086,5.277757
1,1,2.000000,3.000000,4.0,5.0000,6.000000,7.000000,8.000000
10,69,2.000000,3.000000,4.0,5.0000,6.000000,7.000000,8.000000
